In [1]:
import pandas as pd 
import folium
import os
import geohash
from folium.plugins import HeatMap
from sklearn import datasets, linear_model
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime
import numpy as np

In [2]:
RESULTS_DIR = "results"

In [3]:
df = pd.read_csv(os.path.join(RESULTS_DIR, "10_million_with_elevation_geohashes_timestamps.csv"))

In [4]:
df.head()

,Unnamed: 0,value,geohash,geohash5,timestamp
0,0,23.0,xn7tncn498k2,xn7tn,1.540076e+09
1,1,19.0,xnezj2x7rnh7,xnezj,1.540076e+09
2,2,15.0,xneysek46d7w,xneys,1.540076e+09
3,3,16.0,u336qgr9tvzw,u336q,1.540076e+09
4,4,18.0,xneyyr50hk99,xneyy,1.540076e+09


In [5]:
points_geohashes_prec_5 = df.geohash5.value_counts()
points_geohashes_prec_5.mean(), points_geohashes_prec_5.median()

(310.4768877404536, 34.0)

In [6]:
geohashes_for_regression = list(points_geohashes_prec_5.where(lambda count: count >= 50).dropna().index)
len(geohashes_for_regression)

11920

In [7]:
groupped_df = df[df.geohash5.isin(geohashes_for_regression)][["value", "geohash5", "timestamp"]].groupby("geohash5")
len(groupped_df)

11920

In [8]:
def prepare_svr_regresion_model(data):
    data = data.sort_values(by=['timestamp'])
    new_data = pd.DataFrame(data=[[]])
    size = data.shape[0]
    new_data['count'] = size
    
    if size <= 1:
        new_data['regresion'] = None
        new_data['reg_error'] = None
        return new_data
    
    train_size = int(0.8 * size)
    test_size = size - train_size
    
    X_train = np.array(data.timestamp[:train_size]).reshape(-1, 1)
    X_test = np.array(data.timestamp[train_size:]).reshape(-1, 1)
    
    Y_train = np.ravel(np.array(data.value[:train_size]).reshape(-1, 1))
    Y_test = np.ravel(np.array(data.value[train_size:]).reshape(-1, 1))
    
    regr = SVR(kernel='rbf', C=1e3, gamma=0.1)

    regr.fit(X_train, Y_train)
    new_data['regresion'] = regr
    
    Y_pred = regr.predict(X_test)
    new_data['reg_error'] = mean_squared_error(Y_test, Y_pred)
    return new_data

In [9]:
df_after_training = groupped_df.apply(prepare_svr_regresion_model).reset_index()

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
df_after_training.head()